In [1]:
import os
import sys
import csv

from lib.Google_DistMatrix import Google_DistMatrix
from lib.Graph import Graph
from lib.Input import Input
from lib.Format import Format

In [2]:
sys.argv = ['main.py']

In [3]:
# ===================================================================
# =                                                                 =
# =                 Verify commandline arguments                    =
# =                                                                 =
# ===================================================================

# check for correct cmd line arguments ("python3 hamiltonian.py csv_file")
argc = len(sys.argv)
print(f"{argc=}")
if argc == 1: # `python3 main.py`
    filename = "addresses/prospect.csv"
    db_name = "addresses/example_db.csv"
elif argc == 2: # `python3 main.py {address_csv}`
    filename = sys.argv[1]
    db_name = "addresses/example_db.csv"
elif argc == 3: # `python3 main.py {address_csv} {db_csv}`
    filename = sys.argv[1]
    db_name = sys.argv[2]
else:
    print("Format: python3 main.py {address_csv} {db_csv}")
    exit(1)

argc=1


In [4]:
# ===================================================================
# =                                                                 =
# =                      Initialise variables                       =
# =                                                                 =
# ===================================================================
API_KEY = os.environ.get("DISTANCE_MATRIX_API_KEY",'') # API_KEY
DM = Google_DistMatrix(API_KEY) # Initialise Google Distance Matrix API
database = {} # database of addresses and users

# key mappings
i_name = 0
i_zid = 1
i_email = 2
i_phone = 3
i_order_beige = 4
i_order_rose = 5
i_order_shirt = 6
i_order_champ = 7
i_address = 8
i_suburb = 9

In [5]:
# ===================================================================
# =                                                                 =
# =                      Read input from CSV                        =
# =                                                                 =
# ===================================================================
addresses_param_str,address_list = Input.retrieve_addresses(filename)

print(f"{addresses_param_str=}")

addresses_param_str='1+Myrtle+Street,Prospect+NSW+2148|12+Union+St,Luthgow+NSW+2790|11+Sloggetts+Road,"+Norway+NSW+2787"|'


In [6]:
# ===================================================================
# =                                                                 =
# =                 GET matrix from Google Maps API                 =
# =                                                                 =
# ===================================================================

path_matrix = DM.get_matrix(addresses_param_str)
print(f"{path_matrix=}")

if path_matrix['status'] != 'OK':
    print("Error: ", path_matrix['status'])
    exit(1)

find_path_return={'destination_addresses': ['1 Myrtle St, Prospect NSW 2148, Australia', '12 Union St, Lithgow NSW 2790, Australia', '11 Sloggetts Rd, Norway NSW 2787, Australia'], 'origin_addresses': ['1 Myrtle St, Prospect NSW 2148, Australia', '12 Union St, Lithgow NSW 2790, Australia', '11 Sloggetts Rd, Norway NSW 2787, Australia'], 'rows': [{'elements': [{'distance': {'text': '1 m', 'value': 0}, 'duration': {'text': '1 min', 'value': 0}, 'status': 'OK'}, {'distance': {'text': '108 km', 'value': 107656}, 'duration': {'text': '1 hour 37 mins', 'value': 5808}, 'status': 'OK'}, {'distance': {'text': '156 km', 'value': 156310}, 'duration': {'text': '2 hours 16 mins', 'value': 8143}, 'status': 'OK'}]}, {'elements': [{'distance': {'text': '107 km', 'value': 107279}, 'duration': {'text': '1 hour 37 mins', 'value': 5822}, 'status': 'OK'}, {'distance': {'text': '1 m', 'value': 0}, 'duration': {'text': '1 min', 'value': 0}, 'status': 'OK'}, {'distance': {'text': '62.8 km', 'value': 62821}, '

In [7]:
# ===================================================================
# =                                                                 =
# =                 Populate users+address database                 =
# =                                                                 =
# ===================================================================

with open(db_name) as tsv:
    first_line_passed = False
    for line in csv.reader(tsv, dialect="excel-tab"):
        if not first_line_passed:
            first_line_passed = True
            continue
        print(f"{line=}")
        f_name = filename[:-4] # remove ".csv"
        suburb = f_name.split("/")[1].split("_")[0]
    
        # store only if suburb matches file called
        if line[i_suburb] != suburb:
            print(suburb)
            print(line[i_suburb])
            continue
            
        row = {}
        row["name"] = line[i_name]
        row["zid"] = line[i_zid]
        row["email"] = line[i_email]
        row["phone"] = line[i_phone]
        row["order_beige"] = line[i_order_beige]
        row["order_rose"] = line[i_order_rose]
        row["order_shirt"] = line[i_order_shirt]
        row["order_champ"] = line[i_order_champ]
        row["human_address"] = line[i_address]
        row["suburb"] = line[i_suburb]
        
        # get index of human_input
        addr_index = 0
        # print("searching for ", line[i_address])
        for a in address_list:
            if a == line[i_address]:
                break
            addr_index += 1
            
        # if address not found, don't store
        if addr_index == len(address_list):
            # print("address not found")
            # print("addresses_list = ", addresses_list)
            # print("line[i_address] = ", line[i_address])
            continue
        # print(addr_index)
        # print(addresses_list)
        # match that with index of API response
        # print(find_path_return['destination_addresses'])
        row["API_address"] = path_matrix['destination_addresses'][addr_index]
        
        database[path_matrix['destination_addresses'][addr_index]] = row
        
        # print(row)
        

print("======================= DB ======================")
print(database)
print("======================= ========== ======================")

line=['James A', 'z5123456', 'james@gmail.com', '0411111111', 'L', 'M', '', '', '1 Myrtle Street, Prospect NSW 2148', 'prospect']
line=['Alex A', 'z5123456', 'alex@gmail.com', '0422222222', '', 'S', 'L', '', '12 Union St, Luthgow NSW 2790', 'prospect']
line=['Jimbo A', 'z5123456', 'jimbo@gmail.com', '0433333333', '', 'L', '', 'L', '11 Sloggetts Road, Norway NSW 2787', 'prospect']
======================= DB ======================
{}
======================= ========== ======================


In [8]:
# ===================================================================
# =                                                                 =
# =                 Generate Graph for addresses                    =
# =                                                                 =
# ===================================================================
G1 = Graph(path_matrix["destination_addresses"])

# ===================================================================
# =                                                                 =
# =                 Populate graph with data                        =
# =                                                                 =
# ===================================================================

# ===== UNCOMMENT DEPENDING ON DISTANCE OR TIME =====
# G1.graph = matrix_by_distance(path_matrix)
G1.graph = DM.matrix_by_time(path_matrix)
# ===================================================

In [9]:
# ===================================================================
# =                                                                 =
# =               Solve for optimal hamiltonian circuit             =
# =                                                                 =
# ===================================================================
path,trip_time = G1.solve()

# ===================================================================
# =                                                                 =
# =                      Print finalised route                      =
# =                                                                 =
# ===================================================================
Format.print_final_route(path, path_matrix["destination_addresses"], trip_time, database)

==================== FINAL ROUTE =================
ETA =  2 hours, 36 minutes
Number of stops =  3


KeyError: '1 Myrtle St, Prospect NSW 2148, Australia'